In [57]:
import torch
import seaborn as sea
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import random
import datetime
import re
import matplotlib.pyplot as plt
from scipy.stats import poisson
%run functions.ipynb
from unidecode import unidecode
plt.rcParams['figure.figsize'] = [15, 8]
pd.set_option("display.precision", 3)

In [58]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.min_rows', 10)
torch.set_printoptions(sci_mode=False)

In [59]:
opp_stats = pd.read_csv('opponent_stats',index_col=0)
schedule = pd.read_csv('schedule',index_col=0)
data = pd.read_csv('data.csv')

for i in range(1, 20):
    file_name = f"Lines/betting_lines_{i}"
    try:
        # Use globals() to dynamically set the variable name
        globals()[f"lines{i}"] = pd.read_csv(file_name, index_col=0)
    except FileNotFoundError:
        print(f"{file_name} not found, stopping iteration.")
        break

betting_lines_15 not found, stopping iteration.


In [60]:
for i in range(1, 20):
    file_name = f"lines3_{i}"
    try:
        # Use globals() to dynamically set the variable name
        globals()[f"lines3_{i}"] = pd.read_csv(file_name, index_col=0)
        print(file_name)
    except FileNotFoundError:
        print(f"{file_name} not found")


lines3_1 not found
lines3_2 not found
lines3_3 not found
lines3_4 not found
lines3_5 not found
lines3_6 not found
lines3_7 not found
lines3_8
lines3_9
lines3_10
lines3_11
lines3_12
lines3_13 not found
lines3_14
lines3_15
lines3_16
lines3_17
lines3_18
lines3_19


In [61]:
data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
data['Rest'] = data.groupby('Player')['Date'].diff().dt.days
data = data.fillna(5)

In [98]:
def pt_corr(player_a_name, player_b_name, team,stat1='PTS',stat2='PTS'):
    team_games = data.loc[data.Tm==team]
    # extract two columns of interest (i.e. points scored by the two players)
    player_a_dates = set(team_games[team_games['Player'] == player_a_name]['Date'])
    player_b_dates = set(team_games[team_games['Player'] == player_b_name]['Date'])
    common_dates = player_a_dates.intersection(player_b_dates)
    stats = team_games[team_games['Date'].isin(common_dates)]
    
    player_a_points = stats[stats['Player'] == player_a_name][stat1].reset_index(drop=True)
    player_b_points = stats[stats['Player'] == player_b_name][stat2].reset_index(drop=True)
    sample = len(player_a_points)
    # calculate the correlation coefficient between the two columns
    corr_coeff = player_a_points.corr(player_b_points)
    # return the correlation coefficient
    return corr_coeff,sample

In [99]:
team_list = data.Tm.unique()

In [100]:
kings = data.loc[data.Tm=='SAC']
kings.loc[kings.GS==1].Player.unique()

array(["De'Aaron Fox", 'Domantas Sabonis', 'Harrison Barnes',
       'Kevin Huerter', 'Keegan Murray', 'Davion Mitchell',
       'Terence Davis', 'Richaun Holmes', 'Kessler Edwards', 'KZ Okpala',
       'Alex Len'], dtype=object)

In [101]:
top_scorers = data.Player.unique()[0:150]

In [102]:
team2players = {}
for team in team_list:
    team_games = data.loc[data.Tm==team]
    players = team_games.loc[team_games.GS==1].Player.unique()
    team2players[team]=players

In [103]:
team2players['MIL']

array(['Giannis Antetokounmpo', 'Jrue Holiday', 'Brook Lopez',
       'Bobby Portis', 'Grayson Allen', 'Jevon Carter', 'Pat Connaughton',
       'Khris Middleton', 'Jordan Nwora', 'MarJon Beauchamp'],
      dtype=object)

In [104]:

correlate = []
for team in team_list:
    for p in team2players[team]:
        if p in top_scorers:
            for p2 in team2players[team]:
                if p != p2:
                    corr,sample = pt_corr(p,p2,team,stat1='PTS')
                    correlate.append([team,sample,p,p2,'PTS',corr])
                    corr,sample = pt_corr(p,p2,team,stat1='AST')
                    correlate.append([team,sample,p,p2,'AST',corr])
all_corr = pd.DataFrame(correlate,columns=['Team','Sample','P1','P2','Stat1','Corr']).sort_values(by='Corr').dropna()
all_corr = all_corr.loc[all_corr['Sample']>10]
all_corr

C:\Users\User\anaconda3\lib\site-packages\numpy\lib\function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\User\anaconda3\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Team,Sample,P1,P2,Stat1,Corr
3618,CHA,13,LaMelo Ball,Theo Maledon,PTS,-0.703
3765,SAS,14,Jakob Poeltl,Gorgui Dieng,AST,-0.693
3061,NOP,19,Brandon Ingram,Willy Hernangomez,AST,-0.628
3982,DET,12,Killian Hayes,Cade Cunningham,PTS,-0.608
1296,SAC,14,De'Aaron Fox,Kessler Edwards,PTS,-0.585
...,...,...,...,...,...,...
3574,CHA,38,Kelly Oubre Jr.,Terry Rozier,PTS,0.645
3524,CHA,38,Terry Rozier,Kelly Oubre Jr.,PTS,0.645
3572,CHA,13,P.J. Washington,Svi Mykhailiuk,PTS,0.696
301,OKC,21,Jalen Williams,Eugene Omoruyi,AST,0.701


In [105]:
all_corr.tail(10)

,Team,Sample,P1,P2,Stat1,Corr
3977,DET,16,Killian Hayes,James Wiseman,AST,0.612
97,PHI,14,James Harden,Jalen McDaniels,AST,0.617
3813,SAS,25,Devin Vassell,Malaki Branham,AST,0.619
3893,DET,19,Bojan Bogdanovic,Rodney McGruder,AST,0.625
2914,LAL,40,Dennis Schroder,Thomas Bryant,PTS,0.638
3574,CHA,38,Kelly Oubre Jr.,Terry Rozier,PTS,0.645
3524,CHA,38,Terry Rozier,Kelly Oubre Jr.,PTS,0.645
3572,CHA,13,P.J. Washington,Svi Mykhailiuk,PTS,0.696
301,OKC,21,Jalen Williams,Eugene Omoruyi,AST,0.701
1717,TOR,17,Fred VanVleet,Jakob Poeltl,AST,0.757


In [106]:
pd.set_option('display.max_rows', 25)
pd.set_option('display.min_rows', 25)
russ = all_corr.loc[all_corr.P1=='Khris Middleton']
russ

,Team,Sample,P1,P2,Stat1,Corr


In [110]:
all_corr.loc[all_corr.P1== 'Brandon Ingram']

,Team,Sample,P1,P2,Stat1,Corr
3061,NOP,19,Brandon Ingram,Willy Hernangomez,AST,-0.628
3056,NOP,40,Brandon Ingram,Herbert Jones,PTS,-0.436
3048,NOP,12,Brandon Ingram,Zion Williamson,PTS,-0.385
3065,NOP,23,Brandon Ingram,Dyson Daniels,AST,-0.300
3053,NOP,16,Brandon Ingram,Josh Richardson,AST,-0.294
3058,NOP,33,Brandon Ingram,Larry Nance Jr.,PTS,-0.237
3042,NOP,37,Brandon Ingram,CJ McCollum,PTS,-0.217
3060,NOP,19,Brandon Ingram,Willy Hernangomez,PTS,-0.202
3064,NOP,23,Brandon Ingram,Dyson Daniels,PTS,-0.170
3055,NOP,27,Brandon Ingram,Jose Alvarado,AST,-0.084


In [108]:
all_corr.to_csv('correlation',index=0)